In [2]:
import folium
import pandas as pd

In [3]:
# Import data from GeoName
import requests
import json

def getData(place):
    base_url = 'http://api.geonames.org/searchJSON'
    payload = {'username': 'adaisp',
               'country' : 'CH',
               'q' : place,
               'fcodeName' : 'university',
                }
    response = requests.get(base_url, params=payload)
    data = json.loads(response.text)
    try:
        i = 0
        results = {
                'canton': data['geonames'][i]['adminName1'],
                'code': data['geonames'][i]['adminCode1'],
                'lat' : data['geonames'][i]['lat'],
                'long' : data['geonames'][i]['lng'],
            }
        while (data['geonames'][i]['adminName1'] == ''):
            i += 1
            results = {
                'canton': data['geonames'][i]['adminName1'],
                'code': data['geonames'][i]['adminCode1'],
                'lat' : data['geonames'][i]['lat'],
                'long' : data['geonames'][i]['lng'],
            }
    except:
        results = {
            'canton': 'No Match',
            'code': 'No Match',
            'lat' : 'No Match',
            'long' : 'No Match',
        }
        
    return results

In [4]:
grant_export = 'P3_GrantExport.csv'
raw_data = pd.read_csv(grant_export, sep=';', error_bad_lines=False)

uni_data = raw_data.loc[:, ['University', 'Approved Amount']]
uni_data[['Approved Amount']] = uni_data[['Approved Amount']].apply(pd.to_numeric, errors='coerce')
uni_data = uni_data.groupby(['University'])['Approved Amount'].sum()
uni_data.sort_values(ascending=False, inplace=True)

In [5]:
data = pd.DataFrame(uni_data)
data['Records number'] = raw_data['University'].value_counts()
data['Canton'] = ['']*len(data)
data['Code'] = ['']*len(data)
data['Latitude'] = ['']*len(data)
data['Longitude'] = ['']*len(data)

for univ,row in data.iterrows():
    try:
        info = getData(univ.split(' - ')[1])
        if (info.get('canton') == 'No Match'):
            info = getData(univ.split(' - ')[0])
    except:
        info = getData(univ.split(' - ')[0])
    data.set_value(univ, 'Canton', info.get('canton'))
    data.set_value(univ, 'Code', info.get('code'))
    data.set_value(univ, 'Latitude', info.get('lat'))
    data.set_value(univ, 'Longitude', info.get('long'))

After getting informations from GeoNames API, we have to check if these informations cover at least 95% of the records in the raw_data:

In [6]:
total_count = len(raw_data)
canton_found = data[data.Canton != 'No Match']
percentage = canton_found['Records number'].sum() / total_count
percentage

0.7304319279651081

It only covers 73%, so we have to complete it manually.

In [7]:
canton_found

,Approved Amount,Records number,Canton,Code,Latitude,Longitude
University,,,,,,
Université de Genève - GE,1.838237e+09,6394,Geneva,GE,46.20222,6.14569
Universität Zürich - ZH,1.826843e+09,6774,Zurich,ZH,47.36667,8.55
ETH Zürich - ETHZ,1.635597e+09,6153,Zurich,ZH,47.3763,8.54805
Universität Bern - BE,1.519373e+09,5473,Bern,BE,46.94809,7.44744
Universität Basel - BS,1.352251e+09,4746,Basel-City,BS,47.55839,7.57327
Université de Lausanne - LA,1.183291e+09,4092,Zurich,ZH,47.36667,8.55
EPF Lausanne - EPFL,1.175316e+09,4428,Vaud,VD,46.51939,6.56673
Université de Fribourg - FR,4.575262e+08,2079,Fribourg,FR,46.80683,7.15317
Université de Neuchâtel - NE,3.832046e+08,1596,Neuchâtel,NE,46.99179,6.931


In [15]:
# Creating the map !
map_ch = folium.Map(location=[46.73, 8.2], tiles='Stamen Terrain', zoom_start=8)

for univ, row in canton_found.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=univ).add_to(map_ch)

cantons_topojson = 'ch-cantons.topojson.json'
#folium.TopoJson(open(cantons_topojson),
#                'objects.cantons',
#                name='topojson',
#               ).add_to(map_ch)
#map_ch

map_ch.choropleth(
    geo_str=open(cantons_topojson).read(),
    data=canton_found,
    columns=['Code', 'Approved Amount'],
    key_on='objects.cantons',
    threshold_scale=[10e4, 10e5, 10e6, 10e7, 10e8, 10e9],
    fill_color='YlOrRd',
    fill_opacity=0.3,
    line_weight=2,
)
map_ch

AttributeError: 'NoneType' object has no attribute 'get'